<a href="https://colab.research.google.com/github/nrohrbach/sharedmobility_monitoring/blob/main/sharedmobility_monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monitoring GBFS-Feeds on sharemobility.ch

Beschreibung...

In [14]:
import requests
import pandas as pd

In [3]:
url = 'https://gbfs.prod.sharedmobility.ch/v2/gbfs'

In [4]:
url

'https://gbfs.prod.sharedmobility.ch/v2/gbfs'

In [17]:
feeds = requests.get(url).json()

{'systems': [{'id': 'donkey_thun',
   'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/donkey_thun/gbfs'},
  {'id': 'sponticar',
   'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/sponticar/gbfs'},
  {'id': 'carvelo2go',
   'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/carvelo2go/gbfs'},
  {'id': 'bird-kloten',
   'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/bird-kloten/gbfs'},
  {'id': 'lime_winterthur',
   'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/lime_winterthur/gbfs'},
  {'id': 'lime_opfikon',
   'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/lime_opfikon/gbfs'},
  {'id': 'bolt_zurich',
   'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/bolt_zurich/gbfs'},
  {'id': 'bird-platform-partner-jmfleets-bulle',
   'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/bird-platform-partner-jmfleets-bulle/gbfs'},
  {'id': 'pickebike_basel',
   'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/pickebike_basel/gbfs'},
  {'id': 'donkey_neuchatel',
   'url

In [25]:
# Alle GBFS Feeds als Dataframe laden
id = [s['id'] for s in feeds['systems']]
url = [s['url'] for s in feeds['systems']]
gbfsfeeds = pd.DataFrame(list(zip(id, url)),columns =['provider', 'gbfsurl'])
gbfsfeeds

,provider,gbfsurl
0,donkey_thun,https://gbfs.prod.sharedmobility.ch/v2/gbfs/do...
1,sponticar,https://gbfs.prod.sharedmobility.ch/v2/gbfs/sp...
2,carvelo2go,https://gbfs.prod.sharedmobility.ch/v2/gbfs/ca...
3,bird-kloten,https://gbfs.prod.sharedmobility.ch/v2/gbfs/bi...
4,lime_winterthur,https://gbfs.prod.sharedmobility.ch/v2/gbfs/li...
5,lime_opfikon,https://gbfs.prod.sharedmobility.ch/v2/gbfs/li...
6,bolt_zurich,https://gbfs.prod.sharedmobility.ch/v2/gbfs/bo...
7,bird-platform-partner-jmfleets-bulle,https://gbfs.prod.sharedmobility.ch/v2/gbfs/bi...
8,pickebike_basel,https://gbfs.prod.sharedmobility.ch/v2/gbfs/pi...
9,donkey_neuchatel,https://gbfs.prod.sharedmobility.ch/v2/gbfs/do...


## Auswertung pro Provider

In [87]:
providers = ['donkey_thun', 'edrivecarsharing']
providers

['donkey_thun', 'edrivecarsharing']

In [59]:
# Auswertung stattion_status.json
stationstatus = requests.get('https://gbfs.prod.sharedmobility.ch/v2/gbfs/nextbike_ch/station_status').json()
stationstatus = [s['num_bikes_available'] for s in stationstatus['data']['stations']]
VehiclesInStation =sum(stationstatus)
NumberOfStations = len(stationstatus)

# Auswertung free_bike_status.json
freebikesstatus = requests.get('https://gbfs.prod.sharedmobility.ch/v2/gbfs/nextbike_ch/free_bike_status').json()
freebikesstatus = [s['bike_id'] for s in freebikesstatus['data']['bikes']]
FreeBikes = len(freebikesstatus)

In [60]:
FreeBikes

974

In [63]:
for provider in providers:
  print('https://gbfs.prod.sharedmobility.ch/v2/gbfs/' + provider +'/station_status')



  # Auswertung station_status.json
  stationstatus = requests.get('https://gbfs.prod.sharedmobility.ch/v2/gbfs/' + provider +'/station_status').json()
  stationstatus = [s['num_bikes_available'] for s in stationstatus['data']['stations']]
  VehiclesInStation =sum(stationstatus)
  NumberOfStations = len(stationstatus)


JSONDecodeError: ignored

In [70]:
providers

['donkey_thun', 'edrivecarsharing']

In [86]:
d = pd.DataFrame()

for provider in providers:
    temp = pd.DataFrame(
        {
            'name': providers['providername'],
            'Url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/' + providers['providername'] +'/station_status'
        }
    )

    d = pd.concat([d, temp])
d

,name,Url
0,donkey_thun,https://gbfs.prod.sharedmobility.ch/v2/gbfs/do...
1,edrivecarsharing,https://gbfs.prod.sharedmobility.ch/v2/gbfs/ed...


In [89]:
for provider in providers:
  print(provider)

donkey_thun
edrivecarsharing
